# 使用LangChain调用LLM

## 安装LangChain

使用以下命令安装LangChain。由于使用DeepSeek模型，因此要同时安装`langchain-deepseek`。

```python
%pip install langchain langchain-deepseek
```

> ⚠️注意：
> 如果使用的是虚拟环境(例如Conda或者virtualenv)，应该使用魔法命令`%pip install`，而非shell命令`!pip install`，因为前者会将软件
> 包安装至当前环境，而后者会将软件包安装至默认环境。

使用DeepSeek时，要设置环境变量`DEEPSEEK_API_KEY`。使用其它的大模型时，也要在环境变量中设置好对应的API_KEY，比如使用OpenAI的模型时
要设置`OPENAI_API_KEY`。

## Chat Models

大型语言模型（LLMs）是先进的机器学习模型，在文本生成、翻译、摘要、问答等广泛的语言相关任务中表现出色，无需针对每个场景进行特定任务的
微调，是AI程序开发的核心。在LangChain中，LLMs通过聊天模型（Chat Models）接口访问，该接口将消息列表作为输入并返回消息作为输出。

聊天模型提供了额外的功能:

**工具调用** ：许多流行的聊天模型都提供了一个调用 API 的本地工具。此 API 允许开发人员构建丰富的应用程序，使LLMs能够与外部服务、API
和数据库交互。工具调用还可以用于从非结构化数据中提取结构化信息并执行各种其他任务。

**结构化输出** ：一种使聊天模型以结构化格式响应的技术，例如匹配给定模式的 JSON。

**多模态** ：处理文本以外的数据的能力;例如，图像、音频和视频。

### Chat Models 接口

LangChain的聊天模型通过BaseChatModel接口实现， BaseChatModel接口实现了Runnable接口。聊谈模型都已Chat作为前缀，例如ChatDeepSeek、
ChatOpenAI等。

```mermaid
classDiagram
    Runnable <|-- BaseChatModel : implement
    BaseChatModel <|-- ChatDeepSeek 
    BaseChatModel <|-- ChatOpenAI
    BaseChatModel <|-- ChatXXX

    class Runnable {
        <<interface>>
        + invoke()
    }

    class BaseChatModel {
        <<abstract>>
        + invoke()
        + stream()
        + batch()
        + bind_tools()
        + with_structured_output()
    }
```

> ⚠️注意：
> LangChain 也有旧的LLMs的实现，它们不遵循聊天模型接口，而是使用一个以字符串为输入并返回字符串作为输出的接口。这些模型通常没有Chat
> 前缀（例如，Ollama、Anthropic、OpenAI 等）。这些模型实现 BaseLLM 接口，并且可以用“LLM”后缀命名（例如，OllamaLLM、
> AnthropicLLM、OpenAILLM 等）。一般来说，不应该使用这些模型。

### 关键方法

- `invoke`：与聊天模型交互的主要方法。它接受一个消息列表作为输入，并返回一个消息列表作为输出。
- `stream`：在聊天模型生成时对其输出进行流式传输的方法。
- `batch`： 将多个请求批处理到一个聊天模型中，以提高处理效率。
- `bind_tools`：将工具绑定到聊天模型，以便在模型的执行上下文中使用。
- `with_structured_output`：一个包装器，用于包装原生支持结构化输出的模型的`invoke`方法。

### 输入和输出

LLMs通常通过聊天模型接口访问，该接口将消息作为输入并将消息作为输出返回。消息通常与角色相关联（如，“system”、 “human”、 “assistant”
）并包含文本或多模态数据的一个或多个内容块（例如，图像、音频、视频）。

LangChain 支持两种消息格式与聊天模型交互：

1. LangChain 消息格式 ：LangChain 自己的消息格式，默认情况下使用，并由 LangChain 内部使用。
2. OpenAI 的消息格式 ：OpenAI 的消息格式。

## 使用`init_chat_model`来创建LLM

一种方法是调用`init_chat_model`函数来根据传入的参数初始化对应的模型类。然后执行模型的`invoke`方法来进行大模型调用。

In [2]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

model = init_chat_model(model='deepseek-chat', model_provider='deepseek')

messages = [
    SystemMessage("将输入的文字翻译成英语。"),
    HumanMessage("你好。"),
]

message = model.invoke(messages)
print(message.content)

Hello.


## 使用Stream进行输出

如果要流式的输出模型的返回内容，需要循环调用`stream`方法。

In [3]:
for chunk in model.stream(messages):
    print(chunk.content, end='|')

|Hello|.||

## 直接构造LLM类

另一种方式是直接构造一个模型对象。

In [4]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import HumanMessage, SystemMessage

messages = [SystemMessage('请将输入的文字翻译成英文。'),
            HumanMessage('您好。')]

ds_model = ChatDeepSeek(model='deepseek-chat')
message = ds_model.invoke(messages)
print(message.content)

Hello.


## 使用OpenAI格式输入

除了通过Message列表的形式作为输入，还可以以OpenAI的字典格式来作为输入。

In [5]:
from langchain_deepseek import ChatDeepSeek

messages = [
    {'role': 'system', 'content': '请将输入的文字翻译成英文。'},
    {'role': 'user', 'content': '您好。'},
    ]

ds_model = ChatDeepSeek(model='deepseek-chat')
message = ds_model.invoke(messages)
print(message.content)

Hello.


 ## 使用提示词模板

 实际应用过程中，提示词往往都不是固定的，而是会根据具体的情况变化。langchain提供了`ChatPromptTemplate`来应对这种情况。

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', '请将输入的文字翻译成{language}'),
        ('user', '{text}')
    ]
)

prompt = prompt_template.invoke({'language': '日语', 'text': '您好'})

ds_model = ChatDeepSeek(model='deepseek-chat')
ds_model.invoke(prompt)

AIMessage(content='こんにちは', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 11, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '04e71ce8-cd6f-4651-96f5-2616d87a9a57', 'finish_reason': 'stop', 'logprobs': None}, id='run-165bdc31-b9aa-4d8d-b784-626deffac2d7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 5, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})